In [ ]:
import os
from pylab import *
from pysb.core import *
from pysb.bng import *
from pysb.integrate import *
import matplotlib.pyplot as plt
import numpy as np
from pysb.util import alias_model_components
from pysb.simulator import CupSodaSimulator
from pysb.simulator import ScipyOdeSimulator
from pysb.simulator.bng import BngSimulator
import sys
sys.path.append('../')
sys.path.append('/home/ildefog/main/Projects/NERM/')
sys.path.append('/home/ildefog/main/Projects/NERM/param_files/')
from model_files.nerm_pvals import model 
import pandas as pd
from scipy import stats
from pysb.simulator import SimulationResult
import logging
import os
from pysb.logging import setup_logger
from pysb.simulator.cupsoda import CupSodaSimulator
from pysb.simulator.scipyode import ScipyOdeSimulator
from sensitivity_analysis4 import InitialsSensitivity
from pysb.util import update_param_vals
from scipy.stats import norm,uniform, halfnorm
from pysb.tools.sensitivity_analysis import PairwiseSensitivity
os.environ['OMP_NUM_THREADS'] = '1'
from scipy.optimize import curve_fit

In [ ]:
c0 = np.load('../NecroptosisSens/cluster0/necro_pars10k_cluster0_5720_mod.npy')
c1 = np.load('../NecroptosisSens/cluster1/necro_pars10k_cluster1_5720_mod.npy')
c2 = np.load('../NecroptosisSens/cluster2/necro_pars10k_cluster2_5720_mod.npy')
c3 = np.load('../NecroptosisSens/cluster3/necro_pars10k_cluster3_5720_mod.npy')

In [ ]:
print(c2[0])

In [ ]:
mode2_100 = np.load('../../param_files/necro_pars10k_cluster2_5720_mod.npy')[:,14:]
mode2_100 = np.log10(mode2_100)
tspan = np.linspace(0, 1440, 500)

In [ ]:
mode2_100[0]

In [ ]:
print(mode2_100[0])

In [ ]:
clus1 = np.load('../NecroptosisSens/cluster2/necro_pars10k_cluster2_5720_mod.npy')[:,14:]

In [ ]:
n_pars = len(mode2_100)
all_pars = np.zeros((n_pars, len(model.parameters)))

rate_params = model.parameters_rules()
param_values = np.array([p.value for p in model.parameters])
rate_mask = np.array([p in rate_params for p in model.parameters])

for i in range(n_pars):
    par = mode2_100[i]
    param_values[rate_mask] = 10**par
    all_pars[i] = param_values

In [ ]:
list(model.initial_conditions)

In [ ]:
tspan2 = np.linspace(0,1800,500)

In [ ]:
m = model.components
solverwtm1 = ScipyOdeSimulator(model, tspan=tspan2, verbose = True)
mode1wtm1  = solverwtm1.run(param_values=all_pars, num_processors = 20)
mode1df_wtm1 = mode1wtm1.dataframe

In [ ]:
m1 = model.monomers
mlkltot = mode1wtm1.observable(m1.MLKL())
for item in mlkltot.items():
    print(item)
# plt.figure()
# for n in mlkltot: 
#     plt.plot(tspan2/60, mlkltot[n], lw=1.5, 
#              zorder = 1, color = 'red', alpha = 0.5) 
# plt.show()    

In [ ]:
m = model.components
solverwtm1 = ScipyOdeSimulator(model, tspan=tspan2, verbose = True)
mode1wtm1  = solverwtm1.run(initials = {m.FADD(bDD=None, bDED1=None, bDED2=None):310},
                                        param_values=all_pars, num_processors = 20)
mode1df_wtm1kd = mode1wtm1.dataframe

In [ ]:
m = model.components
solverwtm1 = ScipyOdeSimulator(model, tspan=tspan2, verbose = True)
mode1wtm1  = solverwtm1.run(initials = {m.FADD(bDD=None, bDED1=None, bDED2=None):31090},
                                        param_values=all_pars, num_processors = 20)
mode1df_wtm1oe = mode1wtm1.dataframe

In [ ]:
fig, ax = plt.subplots()
for n in range(len(all_pars)):
    plt.plot(tspan2/60, mode1df_wtm1.loc[n]['MLKLa_obs'].iloc[:]/mode1df_wtm1.loc[n]['MLKLa_obs'].iloc[:].max(), lw=1.5, 
             zorder = 1, color = 'red', alpha = 0.5) 
#     plt.plot(tspan2/60, mode1df_wtm1kd.loc[n]['MLKLa_obs'].iloc[:], lw=1.5, 
#              zorder = 1, color =  'blue', alpha = 0.3)
#     plt.plot(tspan2/60, mode1df_wtm1oe.loc[n]['MLKLa_obs'], lw=1.5, 
#              zorder = 1, color = 'green', alpha = 0.5) 
plt.xlabel("Time (hours)", fontsize=15)
plt.ylabel('[pMLKL]', fontsize=15)
plt.xlim(xmin = 0, xmax = 16)
plt.ylim(ymin = 0)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
right_side = ax.spines["right"]
right_side.set_visible(False)
top = ax.spines["top"]
top.set_visible(False)
ax.set_xticks([0, 5, 10, 15, 20, 24])
ax.grid(False)
plt.tight_layout()
# plt.savefig('pmlkl_allpars_clus0_tnf100_noframe.pdf')
plt.show() 

In [ ]:
threshold2 = 2772

In [ ]:
df_threshtimeswt = mode1df_wtm1.groupby('simulation').apply(lambda x: (x > threshold2).idxmax()).apply(lambda x: [y[1] for y in x])
df_threshtimeskd   = mode1df_wtm1kd.groupby('simulation').apply(lambda x: (x > threshold2).idxmax()).apply(lambda x: [y[1] for y in x])
df_threshtimesoe = mode1df_wtm1oe.groupby('simulation').apply(lambda x: (x > threshold2).idxmax()).apply(lambda x: [y[1] for y in x])

In [ ]:
mode2dfn = pd.DataFrame({'WT': df_threshtimeswt['MLKLa_obs'],
                            'KD': df_threshtimeskd['MLKLa_obs'],
                          'OE': df_threshtimesoe['MLKLa_obs'],
                        'species': 'MLKL'})
framesm4 = [mode2dfn]
dfclus4 = pd.DataFrame(data = pd.concat(framesm4), columns = ['species','WT','KD','OE'])
dfm4fadd = pd.melt(dfclus4, id_vars=["species"])

In [ ]:
sns.set(style="whitegrid")
sns.color_palette("hls", 8)
fig, ax = plt.subplots(figsize = (15,10))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
# my_pal = {"WT": "red", "KD": "lightcoral", "OE":"darkred"}
# b = sns.boxplot(x = "value", y ="variable", data = dfclus0, notch = True, width = 0.5, palette = my_pal)
b = sns.swarmplot(x="value", y="variable", data=dfm4fadd, zorder=2)
# b = sns.boxplot(x = "value", y ="variable", data = dfclus0, notch = True, width = 0.5, palette = my_pal)
b.set_xlabel("Time (m)",fontsize=25)
plt.xticks(fontsize = 25)
# b.set_ylabel("Cluster 0 New",fontsize=15)
b.tick_params(labelsize=25)
plt.xlim(xmax = 1800, xmin = 1)
ax.set_xticks([0, 600, 1200, 1800])
plt.tight_layout()
# plt.savefig('pmlkl_cylda20dko_100tnf_mode1_swarm_redcolors.pdf')
plt.show()

In [ ]:
import seaborn as sns #fadd
plt.figure(figsize = (9,6))
sns.stripplot(data=dfm4fadd, y="species", x="value", hue="variable", dodge=True, 
              orient = 'h',linewidth=0.5,size=7)
plt.xlim(xmin = 1, xmax = 1800)
plt.legend(loc = 'lower right')

In [ ]:
for i,t in enumerate(tspan):
    print(i,":",t)

In [ ]:
sim_idx_faddkd = [idx for idx in range(len(all_pars)) if mode1df_wtm1oe['MLKLa_obs'][idx].iloc[312] <= 2772]
print(len(sim_idx_faddkd))

In [ ]:
import seaborn as sns #ciap
plt.figure()
sns.stripplot(data=dfm4, y="species", x="value", hue="variable", dodge=True, 
              orient = 'h',linewidth=0.5,size=7)
plt.xlim(xmin = 350, xmax = 1000)
plt.legend(loc = 'lower right')

In [ ]:
import seaborn as sns
plt.figure()
sns.stripplot(data=dfm4, y="species", x="value", hue="variable", dodge=True, 
              orient = 'h',linewidth=0.5,size=7)
plt.xlim(xmin = 350, xmax = 1000)
plt.legend(loc = 'lower right')

In [ ]:
df_threshtimeskdnew['MLKLa_obs']

In [ ]:
mode2wt = pd.DataFrame({'KD': df_threshtimeskdnew['MLKLa_obs'],
                       'species':'wt'})
dfclus4 = pd.DataFrame(data = pd.DataFrame(mode2wt), columns = ['species','WT'])
dfm4wt = pd.melt(dfclus4, id_vars=["species"])

In [ ]:
plt.figure()
sns.stripplot(data=dfm4wt, y="species", x="value", hue="variable", dodge=True, orient = 'h')
plt.xlim(xmin = 1)
plt.legend(loc = 'lower right')

In [ ]:
for i,t in enumerate(np.linspace(0,1440,500)):
    print(i,":",t)

In [ ]:
sim_idx_a20ko = [idx for idx in range(len(all_pars)) if mode1df_wtm1['MLKLa_obs'][idx].iloc[193] < 2772]
print(len(sim_idx_a20ko))

In [ ]:
with open('necro_5720mode2mid_paramk15tnf100.txt', 'w') as f:
   for p, v in zip(model.parameters, mode1wtm1.param_values[0]):
       f.write('{},{:e}\n'.format(p.name, v))

In [ ]:
def update_param_vals(model, newvals):
    """update the values of model parameters with the values from a dict.
    the keys in the dict must match the parameter names
    """
    update = []
    noupdate = []
    for i in model.parameters:
        if i.name in newvals:
            i.value = newvals[i.name]
            update.append(i.name)
        else:
            noupdate.append(i.name)
    return update, noupdate

In [ ]:
setup_logger(logging.INFO, file_output='necroclusters2large_5720.log', console_output=True)
observable = 'MLKLa_obs'

tspan = np.array([30, 90, 270, 480, 600, 720, 840, 960, 1080, 1200, 1320, 1440])

def likelihood(mlkl_traj):
    #Here we fit a spline to find where we get 50% release of MOMP reporter
    ysim_mlkl_norm = mlkl_traj / np.nanmax(mlkl_traj)
    st, sc, sk = scipy.interpolate.splrep(tspan, ysim_mlkl_norm)
    try:
        t10 = scipy.interpolate.sproot((st, sc - 0.10, sk))[0]
        t90 = scipy.interpolate.sproot((st, sc - 0.90, sk))[0]
    except IndexError:
        t10 = 0
        t90 = 0

    # time of death  = halfway point between 10 and 90%
    td = (t10 + t90) / 2

    return td

def run():
    vals = np.linspace(0.7, 1.3, 5)
    new_params2 = pd.read_csv('necro_5720mode2mid_paramk15tnf100.txt',
        index_col=0, names=['name', 'value']).to_dict('dict')['value']

    update_param_vals(model, new_params2)

    use_cupsoda = False
    if use_cupsoda:
        integrator_opt = {'rtol': 1e-6, 'atol': 1e-6, 'max_steps': 20000,
                          'memory_usage': 'global', 'vol': 1e-19}
        solver = CupSodaSimulator(model, t, verbose=True, gpu=0,
                                  integrator_options=integrator_opt,
                                  )
    else:
        integrator_opt_scipy = {'rtol': 1e-6, 'atol': 1e-6, 'mxstep': 20000}
        solver = ScipyOdeSimulator(model, tspan=tspan, integrator='lsoda',
                                   integrator_options=integrator_opt_scipy)
    sens = InitialsSensitivity(
        solver,
        values_to_sample=vals,
        observable=observable,
        objective_function=likelihood, sens_type='initials')  #

    sens.run('output_{}', out_dir=out_dir)
    sens.create_heatplot(save_name='mode2k15heatinew', out_dir=out_dir)
    InitialsSensitivity.create_boxplot(sens, save_name='mode2k15boxinew', out_dir=out_dir, show = True)
#     sens.create_histogram(save_name='mode2k0boxinew', out_dir=out_dir)
#     sens.create_individual_pairwise_plots(save_name='mode1k12inew', out_dir=out_dir)


# if __name__ == '__main__':
#     out_dir = 'clus0mnec1_5421'
#     run()
#     process()
#     load()


In [ ]:
if __name__ == '__main__':
    model_v = 'necro'
    # all parameter fits from pso
#     p_set = 'all_params_24_part_200_iter_2_bound_0.1_speed.csv'
    params = pd.read_csv('necro_5720mode2mid_paramk15tnf100.txt',
        index_col=0, names=['name', 'value']).to_dict('dict')['value']
#     print(len(params))
    # how many sens analysis to run
    n_sets = 1

    # spacing for sens analysis
    vals = np.linspace(.7, 1.3, 5)

    solver = ScipyOdeSimulator(model, tspan, integrator='lsoda',
                               verbose=True,
                               use_analytic_jacobian=True,
                               compiler='cython',
                               integrator_options={"rtol": 1e-6,
                                                   "atol": 1e-6})
    # The observable of the model
    observable = 'MLKLa_obs'
    sens = InitialsSensitivity(
        values_to_sample=vals,
        observable=observable,
        objective_function=likelihood,
        solver=solver,sens_type = 'initials'
    )
    out_dir = 'mode2_initials_k15'
#     out_dir = p_set.split(',')[0]
#     if not os.path.exists(out_dir):
#         os.mkdir(out_dir)
    run()
#     process()
#     load()

In [ ]:
def process():
    all_df = []
    n = 1
    sens.p_matrix = np.loadtxt(
        os.path.join(out_dir, 'output_{}_p_matrix.csv'))
    sens.p_prime_matrix = np.loadtxt(
        os.path.join(out_dir, 'output_{}_p_prime_matrix.csv'))
    x = [np.array(mat).flatten() for mat in
         sens.sensitivity_multiset]
    df = pd.DataFrame(
        [{'name': i, 'values': v} for i, j in
         zip(sens.index, x) for v in j])
    # df = pd.DataFrame(df, columns=sens.index)
    df['parameter_set'] = n
    all_df.append(df)

    all_df = pd.concat(all_df)
    all_df.to_csv("mode2_initials_k18_test/sens_summary.csv")

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

In [ ]:
plt.close('all')
InitialsSensitivity.create_boxplot(x_axis_label=None, save_name=None,
                                    out_dir=None, 
                                   show=False, infile = 'mode2_initials_k18_test2/mode2k18boxinew.csv',
                                  names = [i[1].name for i in model.initial_conditions])

In [ ]:
np.arange(5) 

In [ ]:
names= [i[1].name for i in model.initial_conditions]
print(names)
print(list(names))
print(list(reversed(names)))
print(list(range(len(names))))

In [ ]:
names= [i[1].name for i in model.initial_conditions]
print(names)
print(list(reversed(names)))
print(reversed(names))

In [ ]:
N = 5
menMeans = (20, 35, 30, 35, -27)
womenMeans = (25, 32, 34, 20, -25)
menStd = (2, 3, 4, 1, 2)
womenStd = (3, 5, 2, 3, 3)
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

In [ ]:
matplotlib.__version__

In [ ]:
!pip install matplotlib

In [ ]:
fig, ax = plt.subplots()

p1 = ax.bar(ind, menMeans, width, yerr=menStd, label='Men')
p2 = ax.bar(ind, womenMeans, width,
            bottom=menMeans, yerr=womenStd, label='Women')

ax.axhline(0, color='grey', linewidth=0.8)
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.set_xticks(ind, labels=['G1', 'G2', 'G3', 'G4', 'G5'])
ax.legend()

# Label with label_type 'center' instead of the default 'edge'
ax.bar_label(p1, label_type='center')
ax.bar_label(p2, label_type='center')
ax.bar_label(p2)

plt.show()

In [ ]:
import seaborn as sns
def load():
    all_df = pd.read_csv("mode2_initials_k18_test/sens_summary.csv", index_col=0)
    subset = False
    if subset:
        all_df = all_df.loc[all_df.name.isin(sample_list)]
        savename_prefix = 'subset'
    # sort by std
    order = all_df.groupby('name')['values'].std().sort_values().index.values
    order = list(reversed(order))
    # print(list(order))
    plot_all = True
    sns.set(font_scale=2)
    if plot_all:
        plt.figure(figsize=(16, 6))
        plt.boxplot(x='name', y='values', data=all_df,
            scale="count", cut=0, order=order)
        plt.xticks(b.get_xticks(), order, rotation=45)
        plt.xlabel("Model")
        plt.tight_layout()
        savename = 'boxplot_all_{}'.format(model_v)
        if subset:
            savename = savename_prefix + savename
        plt.savefig("{}.png".format(savename), bbox_inches='tight')
        plt.savefig("{}.pdf".format(savename), bbox_inches='tight')
        plt.show()
    plot_each = True
    if plot_each:
        sns.catplot(
            x='parameter_set', y='values',
            col="name", col_wrap=4,
            col_order=order,
            data=all_df, kind="violin",
            scale="count", cut=0,
            height=2., aspect=2,  split=True,

        )
        plt.xticks([])
        plt.tight_layout()
        savename = 'violin_each_{}'.format(model_v)
        if subset:
            savename = savename_prefix + savename
        plt.savefig("{}.png".format(savename), bbox_inches='tight')
        plt.savefig("{}.pdf".format(savename), bbox_inches='tight')
        plt.show()